In [5]:
import re


import pandas as pd

# Load the messy data
url = 'https://www.dropbox.com/scl/fi/qxjv1jjosw1eueouv7v1w/messy_data.csv?rlkey=9qu8bsjd4wgi6rz0br4hrudjd&st=9ngzdwui&dl=1'
df = pd.read_csv(url)

# Preview the data
df.head()

df.info()

# Check for missing values
df.isnull().sum()

#check for null salary
null_salary_entries = df['Salary'].isnull().sum()
print("null salary",null_salary_entries)
#check for null age
null_age_entries = df['Age'].isnull().sum()
print("null age",null_age_entries)


# Check for duplicate rows
df.duplicated().sum()

#get number of duplicate rows
duplicate_rows = df.duplicated().sum()
print("duplicate rows=",duplicate_rows)

# Check column names
df.columns


#Fill missing 'Age' with median
df['Age'] = df['Age'].fillna(df['Age'].median())

#drop duplicates
df = df.drop_duplicates()


#count empty emails
emptyemail=df['Email'].isnull().sum()
print("empty email=",emptyemail)

# Number of rows with invalid emails
duplicate_emails = df['Email'].duplicated().sum()
print("duplicate email=",duplicate_emails)

#fix email format

#count invalid emails
invalid_emails = df['Email'].apply(
    lambda x: not re.match(r"[^@]+@[^@]+\.[^@]+", str(x))
).sum()
print("invalid emails=",invalid_emails)

# Keep only professional emails
df['Email'] = df['Email'].apply(lambda x: x if re.match(r"[^@]+@[^@]+\.[^@]+", str(x)) and not re.search(r'gmail|yahoo|hotmail', str(x), re.I) else None)

#drop empty emails
df = df.dropna(subset=['Email'])
#drop duplicate emails
df= df.drop_duplicates(subset='Email')


#drop empty names
df = df.dropna(subset=['Name'])
#clean name field
df['Name'] = df['Name'].str.replace('[^a-zA-Z\s]', '', regex=True).str.strip().str.title()

#standardise join dates

df['Join Date'] = pd.to_datetime(df['Join Date'], errors='coerce',dayfirst=True)
df['Join Date'] = df['Join Date'].dt.strftime('%Y-%m-%d')


#standardisse department names

df['Department'] = df['Department'].str.lower().str.strip()
#map the short names to standard names
department_map = {
    'hr': 'HR',
    'engineering': 'Engineering',
    'eng': 'Engineering',
    'engg': 'Engineering',
    'marketing': 'Marketing',
    'market': 'Marketing',
    'sales': 'Sales',
    'sale': 'Sales',
    'support': 'Support',
    'supp': 'Support'
}

def clean_department(dept):
    dept = str(dept).lower().strip()
    for keyword in department_map:
        if dept.startswith(keyword):
            return department_map[keyword]
    # fill unmatched departments with other
    return 'Other'

df['Department'] = df['Department'].apply(clean_department)


#handle salary noise


# Remove currency symbols or commas
df['Salary'] = df['Salary'].replace('[\$,]', '', regex=True).astype(float)

#Fill missing Salary with median
df['Salary'] = df['Salary'].fillna(df['Salary'].median())

# Remove salary out of range
count = df[(df['Salary'] >= 10000) & (df['Salary'] <= 500000)].shape[0]
print("outlier salary=",count)
df = df[(df['Salary'] >= 10000) & (df['Salary'] <= 500000)]

#save
df.to_csv("cleaned_dataset.csv", index=False)

df.info()
#download
from google.colab import files
files.download("cleaned_dataset.csv")





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11000 non-null  int64  
 1   ID          11000 non-null  object 
 2   Name        8667 non-null   object 
 3   Age         9253 non-null   float64
 4   Email       9731 non-null   object 
 5   Join Date   8808 non-null   object 
 6   Salary      8761 non-null   float64
 7   Department  8745 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 687.6+ KB
null salary 2239
null age 1747
duplicate rows= 291
empty email= 1092
duplicate email= 1548
invalid emails= 3502
outlier salary= 3027
<class 'pandas.core.frame.DataFrame'>
Index: 3027 entries, 5 to 10974
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3027 non-null   int64  
 1   ID          3027 non-null   object 
 2   Name        30

/tmp/ipython-input-5-2443533888.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Email'] = df['Email'].apply(lambda x: x if re.match(r"[^@]+@[^@]+\.[^@]+", str(x)) and not re.search(r'gmail|yahoo|hotmail', str(x), re.I) else None)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>